In [8]:
import cv2
import os
import glob
import numpy as np

# Obtenção das fotos do tabuleiro para calibração das cameras e consequentemente obtenção dos POI

In [14]:
# Define the key to capture the images
capture_key = ord('c')  # Change it to the desired key

# Define the camera indices (0 and 1 for the first and second cameras)
camera_indices = [1, 2]


# Create directories for saving images
image_dirs = ['camera1_images', 'camera2_images']
for dir_name in image_dirs:
    os.makedirs(dir_name, exist_ok=True)
    os.makedirs(dir_name + '_calibrated', exist_ok=True)

# Initialize the cameras
cameras = []
for index in camera_indices:
    camera = cv2.VideoCapture(index)
    cameras.append(camera)

counter = 0
# Start capturing images
while True:
    # Read frames from each camera
    frames = []
    for camera in cameras:
        ret, frame = camera.read()
        frames.append(frame)

    # Display the frames from each camera
    for i, frame in enumerate(frames):
        cv2.imshow(f'Camera {i+1}', frame)

    # Check if the capture key is pressed
    key = cv2.waitKey(1)
    if key == capture_key:
        counter+=1
        # Save the images to respective folders
        for i, frame in enumerate(frames):
            image_path = os.path.join(image_dirs[i], f'image_{i+1}_{counter}.jpg')
            cv2.imwrite(image_path, frame)
            print(f'Image saved: {image_path}')
    # Check if the 'q' key is pressed to quit the program
    if key == ord('q'):
        break

# Release the cameras and close windows
for camera in cameras:
    camera.release()
cv2.destroyAllWindows()

# Obtenção dos POI

In [31]:
def draw_chess_write(camera):
  # termination criteria
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
  # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
  objp = np.zeros((6*8,3), np.float32)
  objp[:,:2] = np.mgrid[0:8,0:6].T.reshape(-1,2)
  # Arrays to store object points and image points from all the images.
  objpoints = [] # 3d point in real world space
  imgpoints = [] # 2d points in image plane.
  images = glob.glob(f'./camera{camera}_images/*.jpg')
  i=1
  for fname in images:
    
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (8,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        name = i
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (8,6), corners2, ret)
        cv2.imshow('img',img)
        cv2.imwrite(f'./camera{camera}_images_calibrated/calib_' + str(name) + '_calib.jpg',img)
        cv2.waitKey(500)
        i+=1
        
  cv2.destroyAllWindows()
  return objpoints, imgpoints, gray

In [32]:
objpoints_1,imgpoints_1,gray_1 = draw_chess_write('1')
objpoints_2,imgpoints_2,gray_2 = draw_chess_write('2')

In [33]:
ret_1, mtx_1, dist_1, rvecs_1, tvecs_1 = cv2.calibrateCamera(objpoints_1, imgpoints_1, gray_1.shape[::-1], None, None)
ret_2, mtx_2, dist_2, rvecs_2, tvecs_2 = cv2.calibrateCamera(objpoints_2, imgpoints_2, gray_2.shape[::-1], None, None)

In [39]:
def get_poi(mtx_1):
    fx = mtx_1[0][0]
    fy = mtx_1[1][1]
    cx = mtx_1[0][2]
    cy = mtx_1[1][2]
    return fx,fy,cx,cy

In [43]:
poi_1 = get_poi(mtx_1)
poi_2 = get_poi(mtx_2)

In [44]:
poi_1

(1003.4126195181994, 1003.7290709255163, 242.6735977649237, 224.61216308299183)

In [45]:
poi_2

(1016.3303474820608, 1018.7269695187592, 292.82651319821827, 247.3039387822884)

# Obtenção dos POR

In [46]:
width = cv2.imread('./camera1_images/image_1_1.jpg').shape[1]
height = cv2.imread('./camera1_images/image_1_1.jpg').shape[0]

In [49]:
stereocalibration_flags = cv2.CALIB_FIX_INTRINSIC
ret, CM1, dist1, CM2, dist2, R, T, E, F = cv2.stereoCalibrate(objpoints_1, imgpoints_1, imgpoints_2, mtx_1, dist_1,
mtx_2, dist_2, (width, height), criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.0001), flags = stereocalibration_flags)

In [60]:
boresight = R
lever_arm = T

In [65]:
print(boresight)

[[ 0.99326416  0.11581963  0.00348201]
 [-0.0025031   0.05149064 -0.99867034]
 [-0.11584492  0.99193474  0.05143371]]


In [64]:
print(lever_arm)

[[ 0.89447156]
 [21.63204216]
 [20.49912873]]
